In [0]:
import numpy as np
import pandas as pd
import re 
import os 
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Model, Sequential 
from keras.layers import Dense, Embedding, Input, Flatten, Embedding, Dropout
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU , SpatialDropout1D, CuDNNLSTM, TimeDistributed, CuDNNGRU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam
from keras.engine.topology import Layer, InputSpec
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow import set_random_seeds
from numpy.random import seed
from nltk import tokenize 

from keras.models import Model
from keras import backend as K

from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras.layers import BatchNormalization, SpatialDropout1D, LSTM


In [0]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#2. Get the file
data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'

#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_sent_punkt.csv')
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_sent_punkt.csv')
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_sent_punkt.csv')

In [0]:
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

In [0]:
train['mal'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION

rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
#words_in_post = train.comment_text.apply(lambda x: len(str(x).split()))
#let_in_post = train.comment_text.apply(lambda x: len(str(x)))

In [0]:
#words_in_post.plot(kind='kde', xlim=[0,600])

In [0]:
max_features = 40000
maxlen = 400
dropout_rate = 0.25
rs = 42
epochs = 4
batch_size = 512
embed_dim = 50
rec_units = 150


seed(rs)
set_random_seed(rs)

In [0]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

Using TensorFlow backend.


In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
class AucPrRecEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = average_precision_score(self.y_val, y_pred)
            print("\n AUC-Precision Recall - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [0]:
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing import text
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)

In [0]:
max_sen_len = 25
max_sent_amount = 35

In [0]:
#from tensorflow.python.client import device_lib

#print(device_lib.list_local_devices())
from keras.layers import GlobalAveragePooling1D
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def make_hat(MAX_SENT_LENGTH = max_sen_len, MAX_SENTS=max_sent_amount, MAX_NB_WORDS=max_features, EMBEDDING_DIM=embed_dim, rec_units=rec_units):

    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    embedded_sequences = Embedding(MAX_NB_WORDS+1, EMBEDDING_DIM, trainable=True)(sentence_input)
    embedded_sequences = SpatialDropout1D(dropout_rate)(embedded_sequences)
    l_lstm = Bidirectional(CuDNNGRU(rec_units, return_sequences=True, kernel_regularizer=l2(0.01)))(embedded_sequences)
    #l_dense = TimeDistributed(BatchNormalization())(l_lstm)
    l_att = AttentionWithContext()(l_lstm)
    sentEncoder = Model(sentence_input, l_att)

    review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    review_encoder = TimeDistributed(sentEncoder)(review_input)
    l_lstm_sent = Bidirectional(CuDNNGRU(rec_units, return_sequences=True,  kernel_regularizer=l2(0.01)))(review_encoder)
    #l_dense_sent = TimeDistributed()(l_lstm_sent)
    l_att_sent = AttentionWithContext()(l_lstm_sent)

    preds = Dense(1, activation='sigmoid')(l_att_sent)
    model = Model(review_input, preds)

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    return model

In [0]:
from sklearn.metrics import average_precision_score, roc_auc_score

def clean_str(string):
    string = string.replace(",", ".").replace(";", ".").replace(":", ".").replace("-", ".")
    return string.strip().lower()

def tok_sentence(s):
    #temp = t.texts_to_sequences(s.replace("?", " ").replace("!", " ").replace(".", " ").split())
    temp = tokenizer.texts_to_sequences(s)
    #print(s)
    if len(temp)==0:
        return np.array([0])
    #else:
    #    temp = np.concatenate(temp).astype(int)
    #if type(temp[0])==list:
    #    temp = [item for sublist in temp for item in sublist]
    return temp#.reshape(-1)  
c = 0
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index] 
    
    train_posts = []
    train_labels = []
    train_texts = []

    #TRAIN
    for i, value in enumerate(X_train):
        if(i%10000==0):
            print(i)
        text = clean_str(value)
        train_texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        sentences = tok_sentence(sentences)
        x = len(sentences)<max_sent_amount
        while x:
            sentences.append(np.array([0])) 
            x = len(sentences)<max_sent_amount

        if len(sentences)>max_sent_amount:
            sentences = sentences[0:max_sent_amount]
        sentences = pad_sequences(sentences, maxlen=max_sen_len)

        train_posts.append(sentences)


    val_posts = []
    val_labels = []
    val_texts = []

    #VAL
    for i, value in enumerate(X_val):
        if(i%10000==0):
            print(i)
        text = clean_str(value)
        val_texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        sentences = tok_sentence(sentences)


        x = len(sentences)<max_sent_amount
        while x:
            sentences.append(np.array([0])) 
            x = len(sentences)<max_sent_amount

        if len(sentences)>max_sent_amount:
            sentences = sentences[0:max_sent_amount]
        sentences = pad_sequences(sentences, maxlen=max_sen_len)
        val_posts.append(sentences)
    
    model = make_hat()
    
    X_train = np.array(train_posts)
    y_train = np.array(y_train)
    X_val =  np.array(val_posts)
    y_val = np.array(y_val)
    
    del train_posts
    del val_posts
    
    
    PrAuc = AucPrRecEvaluation(validation_data=(X_val, y_val), interval=1)
    #filepath="weights_base.best.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
    #early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
    callbacks_list = [PrAuc] #[checkpoint, early, PrAuc]
    
    
    print('Fitting')
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=False,
              validation_data=(X_val, y_val), callbacks = callbacks_list, verbose=1)
    
    probs = model.predict(X_val, batch_size=batch_size, verbose=1)
    auc_f = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    

 fold 0
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0


KeyboardInterrupt: ignored

In [0]:
def clean_str(string):
    string = string.replace(",", ".").replace(";", ".").replace(":", ".").replace("-", ".")
    return string.strip().lower()

def tok_sentence(s):
    #temp = t.texts_to_sequences(s.replace("?", " ").replace("!", " ").replace(".", " ").split())
    temp = tokenizer.texts_to_sequences(s)
    #print(s)
    if len(temp)==0:
        return np.array([0])
    #else:
    #    temp = np.concatenate(temp).astype(int)
    #if type(temp[0])==list:
    #    temp = [item for sublist in temp for item in sublist]
    return temp#.reshape(-1)  



train_posts = []
train_labels = []
train_texts = []

# FULL TRAIN
for i, value in enumerate(X):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    train_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount

    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = pad_sequences(sentences, maxlen=max_sen_len)

    train_posts.append(sentences)

    
test_posts = []
test_labels = []
test_texts = []
    
    
#Test
for i, value in enumerate(X_test):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    test_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount

    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = pad_sequences(sentences, maxlen=max_sen_len)

    test_posts.append(sentences)
    
    
X_train = np.array(train_posts)
y_train = np.array(y)
X_test =  np.array(test_posts)
y_test = np.array(y_test)

del train_posts
del test_posts

In [0]:
model = make_hat()
PrAuc = AucPrRecEvaluation(validation_data=(X_test, y_test), interval=1)
    #filepath="weights_base.best.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
    #early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
callbacks_list = [PrAuc] #[checkpoint, early, PrAuc]

print('Fitting')
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=False,  validation_data=(X_test, y_test), callbacks = callbacks_list, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
from sklearn.metrics import average_precision_score, roc_auc_score
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
print(auc_f)
print(roc_f)

Fitting
Train on 158719 samples, validate on 64830 samples
Epoch 1/4
158719/158719 [==============================] - 372s 2ms/step - loss: 0.3575 - acc: 0.9264 - val_loss: 0.1492 - val_acc: 0.9453

 AUC-Precision Recall - epoch: 1 - score: 0.818996 

Epoch 2/4
158719/158719 [==============================] - 368s 2ms/step - loss: 0.1106 - acc: 0.9580 - val_loss: 0.1492 - val_acc: 0.9460

 AUC-Precision Recall - epoch: 2 - score: 0.836748 

Epoch 3/4
158719/158719 [==============================] - 368s 2ms/step - loss: 0.1017 - acc: 0.9609 - val_loss: 0.1502 - val_acc: 0.9467

 AUC-Precision Recall - epoch: 3 - score: 0.840750 

Epoch 4/4
158719/158719 [==============================] - 369s 2ms/step - loss: 0.0962 - acc: 0.9629 - val_loss: 0.1486 - val_acc: 0.9468

 AUC-Precision Recall - epoch: 4 - score: 0.842811 

64830/64830 [==============================] - 40s 617us/step
0.8428105405317886
0.9676187496437914
